## Stage B fo HDSC
### Appliances Energy Analysis and Predictions
Analysing energy data of appliances on the basis of questions as per asked in Stage B of HDSC 
The dataset is publically available at UCI

In [ ]:
#import required libraries
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#load the dataset
file_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv'
Energy_data = pd.read_csv(file_path)
Energy_data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
#drop the date and lights column
features_df = Energy_data.drop(columns = ['date', 'lights'])

In [4]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

# create a MinMaxScaler Object
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(features_df), columns=features_df.columns)

#next we divide our Energy_data into feature and Target Variables
features = normalised_df.drop(columns=['Appliances'])
Appliances_target = normalised_df['Appliances']

## Question 18

From the dataset, fit a linear model on the relationship betwen the temperature in the living room in Celsius (x =T2) and the temperature outside the building (y=T6). What is the R^2 value in two D.P?

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
x = Energy_data[["T2"]]
y = Energy_data[["T6"]]

#Now, we split our dataset into the training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#creating a Linear Regression Object
lre = LinearRegression()

#fit the model
lre.fit(x_train, y_train)

#predictions
y_hat = lre.predict(x_test)


In [6]:
#R squared Value
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, y_hat)
print('The Rsquared value to two decimal places = ',round(r2_score, 2) )

The Rsquared value to two decimal places =  0.64


In [7]:
#Now, we split our dataset into the training and testing dataset. Recall that we had earlier segmented the features and target variables.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, Appliances_target, test_size=0.3, random_state=42)

In [8]:
#we create a Linear Regression Object
linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(x_train, y_train)

#obtain predictions
predicted_values = linear_model.predict(x_test)

## Question 4

Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [9]:
#this function returns the weight of every feature
def get_weights_df(model, feat, col_name):

  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name] = weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
linear_model_weights

,Features,Linear_Model_Weight
0,rv2,-6.347156e+10
1,RH_2,-4.570000e-01
2,T_out,-3.220000e-01
3,T2,-2.360000e-01
4,T9,-1.900000e-01
5,RH_8,-1.580000e-01
6,RH_out,-7.800000e-02
7,RH_7,-4.500000e-02
8,RH_9,-4.000000e-02
9,T5,-1.600000e-02


Answer: From above RH_2 and RH_1 are the features with lowest and highest weights respectively

## Question 6

What is the Residual Sum of Squares (in two decimal places)?

In [10]:
rss = np.sum(np.square(y_test - predicted_values))
print('The Residual Sum of Squares in two decimal places is', round(rss, 2))

The Residual Sum of Squares in two decimal places is 45.35


## Question 8

What is the Coefficient of Determination (in two decimal places)?

In [11]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
print('The Coefficient of Determination in two decimal places = ',round(r2_score, 2))

The Coefficient of Determination in two decimal places =  0.15


## Question 10

What is the Mean Absolute Error (in two decimal places)?

In [12]:
#the mean absolute error in 2 d.p
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
print('The Mean Absolute Error in two decimal places = ', round(mae, 2))

The Mean Absolute Error in two decimal places =  0.05


## Question 17

What is the Root Mean Squared Error (in three decimal places)?

In [13]:
#RMSE
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print('The RMSE in three decimal places is',round(rmse, 3))

The RMSE in three decimal places is 0.088


## Question 13

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [14]:
#apply Ridge Regression
from sklearn.linear_model import Ridge

#create a Ridge Regression Object
ridge_reg = Ridge(alpha=0.4)

#fit the model to the training dataset
ridge_reg.fit(x_train, y_train)

#obtain predictions
y_hat = ridge_reg.predict(x_test)

In [15]:
# next we calculate RMSE after applying Ridge Regression
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print('The RMSE in three decimal places after applying Ridge regression is',round(rmse, 3))

The RMSE in three decimal places after applying Ridge regression is 0.088


from the two answers, the RMSE is same for both cases 

## Question 2

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [16]:
# train a lasso regression model
from sklearn.linear_model import Lasso

# create a Ridge Regression Object
lasso_reg = Lasso(alpha=0.001)

#fit the model to the training dataset
lasso_reg.fit(x_train, y_train)

#predictions
y_hat = lasso_reg.predict(x_test)

In [17]:
#this function returns the weight of every feature
def get_weights_df(model, feat, col_name):
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name] = weights_df[col_name]
  return weights_df

lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
(lasso_weights_df['Lasso_weight']==0).value_counts()

True     22
False     4
Name: Lasso_weight, dtype: int64

**from above, the non-zero features are 4**

## Question 11

What is the new RMSE with the Lasso Regression(in 3 decimal places)?

In [18]:
#RMSE with Lasso Regression
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
print('The RMSE in three decimal places with Lasso regression = ',round(rmse, 3))

The RMSE in three decimal places with Lasso regression =  0.094
